# Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building a **credit scoring** of a potential customer. A ** credit scoring ** is used to evaluate the ability of a potential borrower to repay their loan.

In [1]:
# libraries cell
import pandas as pd
import nltk

## Open the data file and have a look at the general information. 

In [2]:
try:
    data = pd.read_csv('datasets\credit_scoring_eng.csv') ## local PC path
except:
    data = pd.read_csv('/datasets/credit_scoring_eng.csv') ## jupyter hub path
    
data.info()

data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


## Conclusion

1. Data types: floats have to be converted to ints, losing fractions won't affect quality of data, but the data would be cleaner and processed faster.

2. Amount of values seems to match in almost all columns. There are missing values only in "days_employed" and "total_income". Because there are similar amount of values in both of those columns, I can assume that they are also in matching rows: people don't work, so they don't have an income. I would check this assumption later.

3. Some of "days employed" appear as negative. The value in 4-th row also seems problematic - 340266 days convert to more than 900 years. It seems I would have not only to convert this column, but also to check all of the values. If there are too many strange values and I won't find any reasonable explanation, I won't be able to use this data. But there also seems to be no need to use it anyway, it has nothing to do with the inital task.

4. I see capitalized and lowercase values with the same text. They will be dealt with while looking for duplicates.

5. Loan purposes appear as regular uncategorized text, I will have to process it to extract the keywords, like "purchase", "house", "car", "wedding" and so on.

## Data preprocessing

### Processing missing values

**Notes**

Right now I don't know why there are missing values. I am going to check if 'days_employed' and 'total_income' missing values are in the same rows to see if it has something to do with unemployment.

If the values in 'total income' column are missing in exactly the same row as values in 'days_employed', the bool for it will be True. If they are not, the bool will be False.

In [3]:
# there are 2174 rows wih missing values in each column
data[data['days_employed'].isnull()]['total_income'].isnull().value_counts()

True    2174
Name: total_income, dtype: int64

**Notes**:

So, all the missing values are in same rows, as I assumed.

Now I am going to check what is the employment status in all of these rows and either confirm or disprove my theory.

In [4]:
data[data['total_income'].isnull()]['income_type'].value_counts()

employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64

**Notes**:

They are not unemployed, so I can't just fill missing values with zeroes.

I am going to count global employment statuses and compare them to the count of employment of those whose income is unknow: 

In [5]:
print(data['income_type'].value_counts())
print()
print(data[data['total_income'].isnull()]['income_type'].value_counts() / data['income_type'].value_counts())

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

business                       0.099902
civil servant                  0.100754
employee                       0.099379
entrepreneur                   0.500000
paternity / maternity leave         NaN
retiree                        0.107106
student                             NaN
unemployed                          NaN
Name: income_type, dtype: float64


**Notes**:

I still don't see any explanation for missing values. It can be either human or machine error, no way to know.

The ratio is roughly the same in all major categories, except for 'entrepreneur' which appears only in two rows. 

Also I don't see any reason for the 'entrepreneur' to appear as separate category when I have 'business' category. 
So I am going to replace them with 'business'.

Then I will fill missing incomes basing on mean value by employment status, filling blanks this way should not affect overall data and future conclusions in any meaningful way.

In [6]:
# include 'entrepreneur' into 'business' category
data['income_type'].replace({'entrepreneur':'business'}, inplace = True) 

# fill 'income' blanks with averages by employment category
data['total_income'] = data['total_income'].fillna(data.groupby('income_type')['total_income'].transform('median'))

**Notes**:

Now I want to have a closer look at 'days_employed' column and to see how many corrupted values it has. 

In [7]:
days_employed_negative_counter = 0 # this will help me to count negative values
days_employed_high_values_counter = 0 # and this will help me to count values which are too high for average human lifespan

for value in data['days_employed']: # loop to help me to comb trough data and to count irrational values
    if value < 0:
        days_employed_negative_counter += 1
    if value > (365*50): # I roughly estimated 50 years as maximum employment period
        days_employed_high_values_counter += 1
        
print('Amount of negative values in column:', days_employed_negative_counter)
print('Amount of high values in column:', days_employed_high_values_counter)

Amount of negative values in column: 15906
Amount of high values in column: 3445


**Notes**:

Too many. It is obvious that I can't use data in this column, so I won't fill empty cells there.

I will drop it for good, and also I will extract only the relevant to the research columns to a separate dataframe, and have a look on it.

In [8]:
# create new dataframe
data_extracted = data[[
    'children', 'dob_years', 'education', 'family_status', 'income_type', 'debt', 'total_income', 'purpose'
]].copy()

data_extracted.info()
data_extracted.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   children       21525 non-null  int64  
 1   dob_years      21525 non-null  int64  
 2   education      21525 non-null  object 
 3   family_status  21525 non-null  object 
 4   income_type    21525 non-null  object 
 5   debt           21525 non-null  int64  
 6   total_income   21525 non-null  float64
 7   purpose        21525 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 1.3+ MB


,children,dob_years,education,family_status,income_type,debt,total_income,purpose
0,1,42,bachelor's degree,married,employee,0,40620.102,purchase of the house
1,1,36,secondary education,married,employee,0,17932.802,car purchase
2,0,33,Secondary Education,married,employee,0,23341.752,purchase of the house
3,3,32,secondary education,married,employee,0,42820.568,supplementary education
4,0,53,secondary education,civil partnership,retiree,0,25378.572,to have a wedding


### Conclusion

Now I don't have missing values and my table is tidier and easier to look at. But I have a float column and two int64 columns, I will have to deal with them in the next step.

### Data type replacement

In [9]:
# making a list of names of the columns with numeric values
numeric_column_names = ['children','dob_years','debt','total_income']

# converting those columns to int32 (int16 seems to be to small for some of the values)
data_extracted[numeric_column_names] = data_extracted[numeric_column_names].astype('int32')

data_extracted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   children       21525 non-null  int32 
 1   dob_years      21525 non-null  int32 
 2   education      21525 non-null  object
 3   family_status  21525 non-null  object
 4   income_type    21525 non-null  object
 5   debt           21525 non-null  int32 
 6   total_income   21525 non-null  int32 
 7   purpose        21525 non-null  object
dtypes: int32(4), object(4)
memory usage: 1009.1+ KB


### Conclusion

Now all numerical columns have the same type.

### Processing duplicates

**Notes**:

First of all, I am going to remove capital letters in all categories. Then I will take a closer look at all categorical values, except 'purpose' ('purpose' contains written text and I will leave it for later).

In [10]:
#removing capital letters:
data_extracted['education'] = data_extracted['education'].str.lower()
data_extracted['family_status'] = data_extracted['family_status'].str.lower()
data_extracted['income_type'] = data_extracted['income_type'].str.lower()
data_extracted['purpose'] = data_extracted['purpose'].str.lower()

#looking at values count to check if manual replacement of duplicates is needed:

print('Education')
print('--------------------')
print(data_extracted['education'].value_counts())
print()

print('Family status')
print('--------------------')
print(data_extracted['family_status'].value_counts())
print()

print('Income_type')
print('--------------------')
print(data_extracted['income_type'].value_counts())
print()

Education
--------------------
secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64

Family status
--------------------
married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

Income_type
--------------------
employee                       11119
business                        5087
retiree                         3856
civil servant                   1459
unemployed                         2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64



**Notes**:

No manual replacement of duplicates is needed in those three categories.
Now I will work on duplicates in 'purpose' column. Because it was filled with free written text, I will have to stem it's values first (not lemmatize, because I need common roots for any parts of speech, nouns, verbs and all). Then I will take a look at the list of processed values.

In [11]:
#import natural language toolkit I need to stem all the words
from nltk.stem import SnowballStemmer

english_stemmer = SnowballStemmer('english')

# function for stemming values:
def stem_text(value):
    tokenized = nltk.word_tokenize(value)
    stemmed = ' '.join([english_stemmer.stem(word) for word in tokenized])
    return stemmed

# adding a column with stemmed values
stemmed_values = data_extracted['purpose'].copy().apply(stem_text)

# displaying list of stemmed values:
stemmed_values.unique()

array(['purchas of the hous', 'car purchas', 'supplementari educ',
       'to have a wed', 'hous transact', 'educ', 'have a wed',
       'purchas of the hous for my famili', 'buy real estat',
       'buy commerci real estat', 'buy residenti real estat',
       'construct of own properti', 'properti', 'build a properti',
       'buy a second-hand car', 'buy my own car',
       'transact with commerci real estat', 'build a real estat', 'hous',
       'transact with my real estat', 'car', 'to becom educ',
       'second-hand car purchas', 'get an educ', 'wed ceremoni',
       'to get a supplementari educ', 'purchas of my own hous',
       'real estat transact', 'get higher educ', 'to own a car',
       'purchas of a car', 'profil educ', 'univers educ',
       'buy properti for rent out', 'to buy a car', 'hous renov',
       'go to univers'], dtype=object)

**Notes**:

Looks like I have a lot of values, and many of them still return on themselves. So I plan to make a function, which will merge all the similar values into unified categories, so I will have list of nice and clear purposes without duplicates.

Merged purpose values as I see them:
* Real estate - here I will include everything with 'hous', 'properti' and 'real estat'. There could be also many subcategories here, but, if it will be needed, I can further categorize it later.
* Vehicle - everything related to 'car'
* Wedding - everything about 'wed'
* Education - everything with 'educ' and 'univers'

In [12]:
# write a function as described above:
def merge_purposes(value):
    result = 'unknown'
    words = value.split(' ')
    for word in words:
        stemmed_word = english_stemmer.stem(word)
        if stemmed_word == 'hous' or  stemmed_word == 'properti' or stemmed_word == 'estat':
            result = 'Real estate'
            break
            break  
        elif stemmed_word == 'car':
            result = 'Vehicle'
            break
            break
        elif stemmed_word == 'wed':
            result = 'Wedding'
            break
            break 
        elif stemmed_word == 'educ' or  stemmed_word == 'univers':
            result = 'Education'
            break
            break
        else:
            result = value
    return result


# merge values using the function
data_extracted['purpose'] = data_extracted['purpose'].apply(merge_purposes)

# look at unique values after merging to see if I missed one.
data_extracted['purpose'].value_counts()

Real estate    10840
Vehicle         4315
Education       4022
Wedding         2348
Name: purpose, dtype: int64

Now I don't have duplicate values in all of the categorical value columns, so I can proceed to dropping duplicates.

In [13]:
# checking for number of duplicated rows
data_extracted.duplicated().sum()

576

In [14]:
# dropping duplicated rows. I did not reset indices in case I will need to refer to some row in the initial dataframe 'data'
data_extracted = data_extracted.drop_duplicates() 

data_extracted.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20949 entries, 0 to 21524
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   children       20949 non-null  int32 
 1   dob_years      20949 non-null  int32 
 2   education      20949 non-null  object
 3   family_status  20949 non-null  object
 4   income_type    20949 non-null  object
 5   debt           20949 non-null  int32 
 6   total_income   20949 non-null  int32 
 7   purpose        20949 non-null  object
dtypes: int32(4), object(4)
memory usage: 1.1+ MB


### Conclusion

I removed duplicated string values in all columns. 

In income_type, family_status and education I needed only to convert text to lowercase. 

In 'purpose' I merged values into 4 large categories. To prevent mistakes which could distort my data, I used stemming to find keywords for each of the categories, and sifted through the column, stemming every word and comparing it to my keywords. As a result, every value in 'purpose' was successfully sorted into one of those 4 categories.

As a result of those steps, I was able to find 577 duplicated rows. There could be more than few reasons why those rows was duplicated, for example, same people could ask for loan twice, from two different branches of the same bank or it could be bank clerk's error of some kind.

Finally, I applied remove_duplicates to the dataframe, but didn't reset indices in case there will emerge a need to refer to values from original unprocessed dataframe 'data'.

### Categorizing Data

To answer the question, I need to compare 'debt' to 'children', 'family_status', 'total_income' and 'purpose' columns via pivot tables. Therefore, I need to categorize those columns first.

In the previous steps of the project I already have seen that 'family_status' column already looks OK, it already splits into 5 nice categories. Also while looking for duplicates, I sorted 'purpose' category. So only the three numerical categories need to be adressed to.

In [15]:
# looking at children column
print('Children column')
print('--------------------')
print(data_extracted['children'].value_counts())

# check how many '-1' values out of total and out of '0' do I have.
# it will help me to know if I can to delete these rows or merge '-1' value with '0' without seriously affecting overall data.
print()
print('-1 cells ratio:')
print('--------------------')
print('Cells with -1 value out of total cells: {:.1%}'.format(
    len(data_extracted[data_extracted['children'] == -1]) / len(data_extracted['children'])))
print('Cells with -1 value out of cells with value 0: {:.1%}'.format(
    len(data_extracted[data_extracted['children'] == -1]) / len(data_extracted[data_extracted['children'] == 0])))


# total amount of '-1' values seems to be around 0.2%, which is OK. I can safely drop those rows, but still they may 
# have useful info in other columns. I am going to take a look:

data_extracted[data_extracted['children'] == -1].head()

Children column
--------------------
 0     13707
 1      4719
 2      2023
 3       328
 20       76
-1        47
 4        40
 5         9
Name: children, dtype: int64

-1 cells ratio:
--------------------
Cells with -1 value out of total cells: 0.2%
Cells with -1 value out of cells with value 0: 0.3%


,children,dob_years,education,family_status,income_type,debt,total_income,purpose
291,-1,46,secondary education,civil partnership,employee,0,16450,Education
705,-1,50,secondary education,married,civil servant,0,22061,Vehicle
742,-1,57,secondary education,married,employee,0,10282,Education
800,-1,54,secondary education,unmarried,retiree,0,13806,Education
941,-1,57,secondary education,married,retiree,0,18962,Vehicle


**Notes**:

Now I am going to look at ratio of those rows with 20 children.
I assume it could be inner designation value of some kind, most probably because those families have many children and clerk didn't know how many. Also it could be a typo with extra zero, but I don't think so because:

* 76 is a lot of values for ocassional typo of exactly the same kind.
* I don't see any additional typos like '10' or '30'.

So I will check ratios of those '20' values and compare them to ratios of '0', '2' and children overall to see where it can fit without distorting research results in any significant way.

By the way, for this reasons exactly I also didn't consider '-1' to be a typo, I believe it must be some inner designation with unknown meaning. Maybe it was about families planning children or something like that. If this was real assignment of real client, I would ask about those meanings, but here I don't have such an option. 

In [18]:
print('20 cells ratio:')
print('--------------------')
print('Cells with value "20" out of total cells: {:.1%}'.format(
    len(data_extracted[data_extracted['children'] == 20]) / len(data_extracted['children'])))
print('Cells with value "20" out of cells with value "2": {:.1%}'.format(
    len(data_extracted[data_extracted['children'] == 20]) / len(data_extracted[data_extracted['children'] == 2])))
print('Cells with value "20" out of cells with any number of children: {:.1%}'.format(
    len(data_extracted[data_extracted['children'] == 20]) / len(data_extracted[data_extracted['children'] > 0])))

# It is very small percentage of overall data, and quite small percentage of rows that contain any number of children.
# I am going to take a look at that rows to see if there is something else that stands out:

data_extracted[data_extracted['children'] == 20].head()

20 cells ratio:
--------------------
Cells with value "20" out of total cells: 0.4%
Cells with value "20" out of cells with value "2": 3.8%
Cells with value "20" out of cells with any number of children: 1.1%


,children,dob_years,education,family_status,income_type,debt,total_income,purpose
606,20,21,secondary education,married,business,0,23253,Real estate
720,20,44,secondary education,married,business,0,18079,Real estate
1074,20,56,secondary education,married,employee,1,36722,Education
2510,20,59,bachelor's degree,widow / widower,employee,0,42315,Real estate
2941,20,0,secondary education,married,employee,0,31958,Vehicle


**Notes**:

One of these rows has no age, but age anyway is not relevant to questions. And all the other data I need looks fine there and there is no reason to drop it. 

I would prefer to split those 'rogue' values by categories. To define what categories I need, first I am going to check, how much in percent are there people with every amount of kids out of total. I can see that there are relatively little number of those with 4 or 5 kids, but how little?

In [17]:
(data_extracted['children'].value_counts() / len(data_extracted) * 100).sort_values(ascending = False)

 0     65.430331
 1     22.526135
 2      9.656786
 3      1.565707
 20     0.362786
-1      0.224354
 4      0.190940
 5      0.042961
Name: children, dtype: float64

**Notes**:

So I have total 1.57% of people with 3 kids, 0.19% of people with 4 kids and 0.04% of people with 5 kids, which make total of 1.8%. It is still too small a percentage for separate category, so I will unite them into one category with those who have 2 children. My main categories will be:
* No children (65.43%)
* 1 child (22.53%)
* 2 children or more (11.46%)

Based on my previous assumption, '-1' can be fitted into 'No children' category, but I don't know what number of children is '20' referring to: one or more. So I will replace it with average amount of children (not including 0 and '-1') by family status. 

In [19]:
# check average amount of children by family status (only in rows with children), and by the way converting it to whole number
children_average = data_extracted[data_extracted['children'] > 0].groupby('family_status')['children'].median().astype('int')
print(children_average)

# seems like median is 1 for all categories, so I safely proceed:

family_status
civil partnership    1
divorced             1
married              1
unmarried            1
widow / widower      1
Name: children, dtype: int32


In [20]:
# seems like median is 1 for all categories, so I safely proceed.
# Nice quick function to sort trough those 'children' values:
def has_children(value):
    if value <= 0:
        return 'No children'
    elif value < 2 or value == 20:
        return '1 child'
    else:
        return '2 children or more'
    
# create new 'has_children' column using the function:
data_extracted['has_children'] = data_extracted['children'].copy().apply(has_children)

# look at the result:
print(data_extracted['has_children'].value_counts())

No children           13754
1 child                4795
2 children or more     2400
Name: has_children, dtype: int64


In [21]:
# take a look at debt column:
print(data_extracted['debt'].value_counts())

# looks great.

0    19215
1     1734
Name: debt, dtype: int64


In [22]:
# Now I have a look at income values before I start to categorize them.
print('Ten lowest values')
print('--------------------')
print(data_extracted['total_income'].sort_values().head(10))
print()
print('Ten highest values')
print('--------------------')
print(data_extracted['total_income'].sort_values(ascending = False).head(10))
print()
print('Total income mean:',data_extracted['total_income'].mean())
print('Total income median:',data_extracted['total_income'].median())

Ten lowest values
--------------------
14585    3306
13006    3392
16174    3418
1598     3471
14276    3503
10881    3595
18509    3815
9070     3913
10068    4036
12052    4049
Name: total_income, dtype: int32

Ten highest values
--------------------
12412    362496
19606    352136
9169     276204
20809    274402
17178    273809
17503    255618
18368    248184
18353    228469
15268    216039
11071    205804
Name: total_income, dtype: int32

Total income mean: 26539.67817079574
Total income median: 22880.0


**Notes**:

I don't see any weird values like negatives, and also I don't see people with zero income, which is good. I don't have to process this column before creating income categories based on it. 

Median value is lower than mean, and I see some extremely high difference in income in top 10 values. So I am going to create 4 categories of income based on median value: low, average, above average, high. 

I already know what median is. low and average would be below median, and above average and high would be above.

Now I will find out median of the values lower than overall median, and median of values higher than overall median.

In [23]:
total_median = data_extracted['total_income'].median()
low_values_median = data_extracted[data_extracted['total_income'] <= total_median]['total_income'].median()
high_values_median = data_extracted[data_extracted['total_income'] > total_median]['total_income'].median()

# build sorting function based on calculated values:
def income_level(value):
    result = ''
    if value <= low_values_median:
        result = 'low income'
    elif value <= total_median:
        result = 'average income'
    elif value <= high_values_median:
        result = 'above average income'
    else:
        result = 'high income'
    return result

# creating new column, categorizing rows by income levels:
data_extracted['income_level'] = data_extracted['total_income'].apply(income_level)

#take a look at a new column:
print(data_extracted['income_level'].value_counts())

average income          5238
above average income    5238
low income              5238
high income             5235
Name: income_level, dtype: int64


### Conclusion

To check relation between income and having children, total income, purpose and family status I need to categorize my table by all of those 5 categories. Family status and debt columns were already in perfect condition as it is. Purpose column was processed while looking for duplicates and also was sorted by categories. So I had to deal only with children and total_income columns.

Because number of children doesn't have to do anyting with the initial question, only the fact of having (or not having) kids, I sorted this column into 2 main categories: those who have kids and those who don't.

Income column needed different approach, because I needed a way to assess income levels. I decided to build my categories based on median values because this way I would get more accurate data. So I splitted income column into 4 main categories using median, and median values of values below and above the main median. 

## Answer these questions

- Is there a relation between having kids and repaying a loan on time?

In [24]:
# create a pivot table which summarizes, how many people with and without children did or did not default a loan
# sum counts only '1' values, which correspond only for those in category who defaulted a loan,
# while count corresponds to all the values in the category
pivot_children = data_extracted.pivot_table(
    index = 'has_children',
    values = 'debt',
    aggfunc = [sum,'count']
)

# add a column which represents ratio of those who defaulted a loan in overall category 
pivot_children['ratio in %'] = pivot_children['sum']/pivot_children['count']*100

pivot_children

,sum,count,ratio in %
,debt,debt,
has_children,,,
1 child,450,4795,9.384776
2 children or more,225,2400,9.375000
No children,1059,13754,7.699578


### Conclusion

People who have kids are more likely to default a loan, which seems reasonable: kids are major expence factor, especially in everything connected to unexpected expences. 

Also no matter how many children does the person have, his chance to default a loan stays the same.

- Is there a relation between marital status and repaying a loan on time?

In [25]:
# create pivot table, exactly like in previous step, but with "family_status" as index.
pivot_family = data_extracted.pivot_table(
    index = 'family_status',
    values = 'debt',
    aggfunc = ['sum','count']
)

# add a column which represents ratio of those who defaulted a loan in overall category 
pivot_family['ratio in %'] = pivot_family['sum']/pivot_family['count']*100

pivot_family.sort_values(by = 'ratio in %',ascending = False)

,sum,count,ratio in %
,debt,debt,
family_status,,,
unmarried,273,2770,9.855596
civil partnership,387,4100,9.439024
married,926,11945,7.752198
divorced,85,1192,7.130872
widow / widower,63,942,6.687898


### Conclusion

We can see that unmarried people and people in civil partnership are most likely to default a loan, as opposed to widowers who are least likely to do so. It looks like something related to overall relationship status phases in life: first a person is single, then starts a relationship and moves in, then married, then - if not lucky - either gets divorced or widowed, and with every step a pereson gets least likely to default a loan. Maybe because an average person gets more financially stable as he/she proceeds in life.

- Is there a relation between income level and repaying a loan on time?

In [26]:
# create pivot table, exactly like in previous steps, but with "income_level" as index.
pivot_income = data_extracted.pivot_table(
    index = 'income_level',
    values = 'debt',
    aggfunc = ['sum','count']
)

# add a column which represents ratio of those who defaulted a loan in overall category 
pivot_income['ratio in %'] = pivot_income['sum']/pivot_income['count']*100

pivot_income.sort_values(by = 'ratio in %',ascending = False)

,sum,count,ratio in %
,debt,debt,
income_level,,,
average income,499,5238,9.526537
above average income,452,5238,8.629248
low income,410,5238,7.827415
high income,373,5235,7.125119


### Conclusion

People with average and above average income are more likely to default a loan, than people who have low or high income.

- How do different loan purposes affect on-time repayment of the loan?

In [27]:
# create pivot table, exactly like in previous steps, but with "purpose" as index.
pivot_purpose = data_extracted.pivot_table(
    index = 'purpose',
    values = 'debt',
    aggfunc = ['sum','count']
)

# add a column which represents ratio of those who defaulted a loan in overall category 
pivot_purpose['ratio in %'] = pivot_purpose['sum']/pivot_purpose['count']*100

pivot_purpose.sort_values(by = 'ratio in %',ascending = False)

,sum,count,ratio in %
,debt,debt,
purpose,,,
Vehicle,401,4247,9.441959
Education,370,3937,9.398019
Wedding,185,2289,8.082132
Real estate,778,10476,7.426499


# Conclusion

People who want to take loan to buy a vehicle or to get an education are more likely to default it. It may be connected to job opportunities: those who don't have a vehicle or don't have good profession have less job opportunities, ergo - they are less stable financially.

## General conclusion

In this project I received datasheet of bank's clients and had to evaluate how their family status, number of children, income level and purpose of taking a loan can affect their ability to repay it on time.

I had to process data, dispose of duplicates and somehow deal with irrational values without losing any meaningful data in the process. To do so, I had to decide which columns are most relevant to my research and which are not. In those relevant to it I filled irrational values based on median value by some category, which seemed most rational to me.

Hunting for duplicates in freely worded field 'purpose' was most challenging. First, I had to decide on categories. As I know from real life experience, there is major difference between purchasing commercial and private property, and at first I thought to divide "real estate" category into two: "commercial real estate" and "private real estate". But there was too many values related to real estate where I didn't have any way to know if it was commercial or private, so I had to drop this idea and to leave "real estate" as one major category.

Categorizing was also quite interesting, especially to find categories for "total_income". I considered few different approaches, one of them was to categorize income by procentiles (procentile ten, procentile twenty etc.), but then I decided to concentrate on four main categories and not to complicate things too much. 

By the end of my study I could separate categories of people who are more and less likely to default a loan, as following:

Categories of people who are less likely to default a loan:
* Widowed/Divorced
* With no children
* Either have low or high income
* Need a loan to buy a property

Categories of people who are at risk of defaulting a loan:
* Single
* Parents to children
* Have average or above average income
* Need a loan to get a car or an education

Most results were expected and rational, but they also contained one unexpected which I couldn't find explanation to: surprisingly, people with average and above average income are more likely to default a loan than people whose incomes are low or high. 